In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision

import cv2
from albumentations import Compose, Resize
from albumentations.augmentations.transforms import Normalize
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from IPython.display import display

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 1. data exploring

In [ ]:
train_dataset = pd.read_csv('../input/bengaliai-cv19/train.csv')
print(train_dataset.describe())
display(train_dataset.head())
display(train_dataset.tail())

In [ ]:
unique = train_dataset.apply(lambda col: col.nunique())
unique

In [ ]:
test_dataset = pd.read_csv('../input/bengaliai-cv19/test.csv')
print(test_dataset.describe())
test_dataset.head()

## 2.splitting fold with https://www.kaggle.com/haqishen/validation-with-unseen

In [ ]:
grapheme2idx = {grapheme: idx for idx, grapheme in enumerate(train_dataset.grapheme.unique())}
train_dataset['grapheme_id'] = train_dataset['grapheme'].map(grapheme2idx)

n_fold = 5
skf = StratifiedKFold(n_fold)
for i_fold, (train_idx, val_idx) in enumerate(skf.split(train_dataset, train_dataset.grapheme)):
    train_dataset.loc[val_idx, 'fold'] = i_fold
train_dataset['fold'] = train_dataset['fold'].astype(int)

train_dataset['unseen'] = 0
train_dataset.loc[train_dataset.grapheme_id >= 1245, 'unseen'] = 1
print(train_dataset.unseen.value_counts())

# usage 
fold = 1
train_idx = np.where((train_dataset['fold'] != fold) & (train_dataset['unseen'] == 0))[0]
valid_idx = np.where((train_dataset['fold'] == fold) | (train_dataset['unseen'] != 0))[0]
display(train_dataset.loc[train_idx].reset_index(drop=True).head())
display(train_dataset.loc[valid_idx].reset_index(drop=True).head())

## 3.Dataloader

In [ ]:
!mkdir img; ls

In [ ]:
HEIGHT = 137
WIDTH = 236

def make_png(path):
    df = pd.read_parquet(path)
    data = 255 - df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
    for idx in tqdm(range(len(df))):
        name = df.iloc[idx,0]
        img = (data[idx]).astype(np.uint8)
        cv2.imwrite("/kaggle/working/img/{}.png".format(name),img)

In [ ]:
for path in ['/kaggle/input/bengaliai-cv19/train_image_data_0.parquet',
            '/kaggle/input/bengaliai-cv19/train_image_data_1.parquet',
            '/kaggle/input/bengaliai-cv19/train_image_data_2.parquet',
            '/kaggle/input/bengaliai-cv19/train_image_data_3.parquet']:
    print("now translating: {}".format(path))
    make_png(path)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/working/img'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

In [ ]:
class BengaliImageDataset(Dataset):
    def __init__(self, data_frame, img_path, labels, transform=None):
        super().__init__()
        self.data = data_frame
        self.data_dummie_labels = pd.get_dummies(
            self.data[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']],
            columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
        )
        self.img_path = img_path
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name = os.path.join(self.img_path, self.data.loc[idx, 'image_id'] + '.png')
        img = cv2.imread(image_name)

        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']
            

        if self.labels:
            return {
                'image': img,
                'l_graph': torch.tensor(self.data_dummie_labels.iloc[idx, 0:168]),
                'l_vowel': torch.tensor(self.data_dummie_labels.iloc[idx, 168:179]),
                'l_conso': torch.tensor(self.data_dummie_labels.iloc[idx, 179:186]),
            }
        else:
            return {'image': img}
        
TRAIN_Dataset = BengaliImageDataset(
    data_frame = train_dataset.loc[train_idx].reset_index(drop=True),
    img_path = '/kaggle/working/img/',
    transform = Compose([Resize(HEIGHT,HEIGHT),Normalize(),ToTensorV2()]), labels=True
)
print(len(TRAIN_Dataset))
print(next(iter(TRAIN_Dataset)))
print(next(iter(TRAIN_Dataset))['image'].size())
plt.imshow(next(iter(TRAIN_Dataset))['image'].numpy().T)

In [ ]:
VALID_Dataset = BengaliImageDataset(
    data_frame = train_dataset.loc[valid_idx].reset_index(drop=True),
    img_path = '/kaggle/working/img/',
    transform = Compose([Resize(HEIGHT,HEIGHT),Normalize(),ToTensorV2()]), labels=True
)

batch_size = 32
TRAIN_DataLoader = DataLoader(TRAIN_Dataset, batch_size=batch_size, shuffle=True)
VALID_DataLoader = DataLoader(VALID_Dataset, batch_size=batch_size, shuffle=False)
dataloaders_dict = {"train":TRAIN_DataLoader, "valid":VALID_DataLoader}

## 4.NN model

In [ ]:
print(torchvision.models.resnet50(pretrained=False, progress=True))

In [ ]:
class ResNetFC(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.resnet50(pretrained=False, progress=True)
        self.resnet.load_state_dict(torch.load("../input/pretrained-pytorch-models/resnet50-19c8e357.pth"))
        in_features = self.resnet.fc.out_features
        
        self.fc_graph = torch.nn.Linear(in_features, 168)
        self.fc_vowel = torch.nn.Linear(in_features, 11)
        self.fc_conso = torch.nn.Linear(in_features, 7)
        
    def forward(self, x):
        x = self.resnet(x)
        fc_graph = self.fc_graph(x)
        fc_vowel = self.fc_vowel(x)
        fc_conso = self.fc_conso(x)
        return fc_graph, fc_vowel, fc_conso

In [ ]:
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
model = ResNetFC().to(device)

In [ ]:
update_params_list = ['resnet.fc.weight','resnet.fc.bias',
                      'fc_graph.weight','fc_graph.bias',
                      'fc_vowel.weight','fc_vowel.bias',
                      'fc_conso.weight','fc_conso.bias']
params_to_update = []
for name, param in model.named_parameters():
    if name in update_params_list:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=params_to_update, lr=1e-4)

## 5.Training

In [ ]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    train_loss = []
    valid_loss = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        for phase in ['train', 'valid']:
            if phase == 'train': model.train()
            else: model.eval()
                
            epoch_loss = 0.0
            iteration = 0
            length = len(dataloaders_dict[phase].dataset)

            for batch in dataloaders_dict[phase]:
                iteration += 1
                optimizer.zero_grad()
                inputs = batch["image"]
                l_graph, l_vowel, l_conso = batch["l_graph"], batch["l_vowel"], batch["l_conso"]
                
                # send to device
                inputs = inputs.to(device, dtype=torch.float)
                l_graph = l_graph.to(device, dtype=torch.float)
                l_vowel = l_vowel.to(device, dtype=torch.float)
                l_conso = l_conso.to(device, dtype=torch.float)
                
                with torch.set_grad_enabled(phase == 'train'):
                    # Forward
                    out_graph, out_vowel, out_conso  = model(inputs)
                    loss = criterion(out_graph,l_graph) + criterion(out_vowel, l_vowel) + criterion(out_conso, l_conso)
                    
                    # Backprop
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    batch_loss = loss.item() * inputs.size(0)
                    epoch_loss += batch_loss
                if iteration%50 == 1:
                    print('{} : Minibatch {}/{} finished (Loss: {:.4f})'.format(datetime.datetime.now(),
                                                            min(batch_size*iteration,length),length, batch_loss/batch_size))
        
            epoch_loss = epoch_loss / length
            if phase == 'train':
                train_loss.append(epoch_loss)
            else:
                valid_loss.append(epoch_loss)
            print('##### {} Loss: {:.4f} #####'.format(phase, epoch_loss))
            
        save_path = '/kaggle/working/weights_epoch{}.pth'.format(epoch+1)
        #torch.save(model.state_dict(), save_path)
        
    return model, train_loss, valid_loss

In [ ]:
num_epochs = 3
model, train_loss, valid_loss = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

## 6.Prediction

In [ ]:
for path in ['/kaggle/input/bengaliai-cv19/test_image_data_0.parquet',
            '/kaggle/input/bengaliai-cv19/test_image_data_1.parquet',
            '/kaggle/input/bengaliai-cv19/test_image_data_2.parquet',
            '/kaggle/input/bengaliai-cv19/test_image_data_3.parquet']:
    print("now translating: {}".format(path))
    make_png(path)

In [ ]:
class testDataset(Dataset):
    def __init__(self, data_frame, img_path, transform=None):
        super().__init__()
        self.data = data_frame
        self.img_path = img_path
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name = os.path.join(self.img_path, self.data.loc[idx, 'image_id'] + '.png')
        img = cv2.imread(image_name)

        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']

        return img
    
test_dataset = pd.read_csv('../input/bengaliai-cv19/test.csv')
test_dataset = pd.DataFrame(test_dataset['image_id'].unique(),columns=['image_id'])

TEST_Dataset = testDataset(
    data_frame = test_dataset,
    img_path = '/kaggle/working/img/',
    transform = Compose([Resize(HEIGHT,HEIGHT),Normalize(),ToTensorV2()])
)
TEST_DataLoader = DataLoader(TEST_Dataset, batch_size=1, shuffle=True)

In [ ]:
def predict(model, dataloader, submission):
    prediction = []
    model.to(device)
    model.eval()
    for inputs in dataloader:
        inputs = inputs.to(device, dtype=torch.float)
        with torch.set_grad_enabled(False):
            out_graph, out_vowel, out_conso  = model(inputs)
            _, pred_graph = torch.max(out_graph, 1)
            _, pred_vowel = torch.max(out_vowel, 1)
            _, pred_conso = torch.max(out_conso, 1)
            prediction.append(pred_conso.item())
            prediction.append(pred_graph.item())
            prediction.append(pred_vowel.item())
            
                
    print(prediction)
    for i, pred in enumerate(prediction):
        submission.loc[i,'target'] = pred
        
    return submission

submission = pd.read_csv('../input/bengaliai-cv19/test.csv')[['row_id']].assign(target=0)
submission = predict(model, TEST_DataLoader, submission)
display(submission)

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index = False)

In [ ]:
!rm -r img; ls